In [ ]:
%matplotlib widget
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0,'../src/')
import pyconet as pcn

In [ ]:
plt.rcParams.update({
    "font.family": "sans-serif",
    #"font.sans-serif": "Helvetica",
    "font.sans-serif": "Times New Roman",
})

# Get data from scopus

In [ ]:
author_pubs = pcn.get_scopus_data(query = "7004078700", is_name=False)
pandas_df = pd.DataFrame(author_pubs)
pandas_df

In [ ]:
authors_to_exclude = ['Induja, P.', ]
authors_to_emphasize = ['Marwan, Norbert', 'Kurths, Jürgen', 'Bedard, Michael J.',  'Lieuwen, Tim C.', 'Boxx, Isaac','Chterev, Ianko', 'Anderson, William E.', 'Chaudhuri, Swetaprovo', 'Polifke, Wolfgang', 'Saha, Abhishek', 'Gejji, Rohan', 'Surovyatkina, Elena', 'Juniper, Matthew P.', 'García-Morales, Vladimir', 'Harvazinski, Matthew E.','Emerson, Benjamin', 'Bauch, Chris T.', 'Rywik, Marcin', 'Blumenthal, Ralf S.',]

authors_special = ['Sujith, R. I.']

In [ ]:
author_sid_all, author_name_all, author_sid_to_name_dict_all, author_name_to_sid_dict_all = pcn.create_author_scopus_data_dictionary(author_pubs)

author_sid_to_name_dict, author_name_to_sid_dict = pcn.create_scopus_id_and_name_dictionary(author_sid_to_name_dict_all)

author_sid_to_npub_dict, author_sid_to_npub_sorted = pcn.get_author_sid_number_of_publications_dictionary_with_sorted(author_sid_to_name_dict)

author_sid_to_lid_dict, author_lid_to_sid_dict = pcn.get_scopus_id_to_local_id_dictionary(author_sid_to_name_dict)

coauthor_network = pcn.get_coauthor_network_with_local_index_of_row_col(author_pubs, author_sid_to_name_dict_all, author_sid_to_lid_dict)

number_of_collaborations = np.sum(coauthor_network, axis=1)

Na_per_cir, Na_ids, Na_ide = pcn.get_author_distribution_over_circles(len(author_sid_to_lid_dict), power=4)

radius_arr = pcn.get_circle_rradius(N_circles= len(Na_per_cir), R_max=10, R1_offset = 20) 

x_arr, y_arr, author_sid_to_xy = pcn.get_author_scopus_index_to_xy(radius_arr, Na_ids, Na_ide, author_sid_to_npub_sorted)

author_sid_to_xy = pcn.update_authors_to_emphasize(author_sid_to_xy, R_emph=20, authors_to_emphasize= authors_to_emphasize, author_name_to_sid_dict= author_name_to_sid_dict)

author_sid_to_xy = pcn.update_authors_special(author_sid_to_xy, author_name_to_sid_dict, authors_special, x_shift=0, y_shift=10)

In [ ]:
fig = plt.figure(figsize=(10,7.5))
axs = fig.add_subplot(1,1,1)
axs.axis('off')
axs.plot([0,0], [1,1], 'w')
axs.set_aspect(aspect=.7)

color_var_arr = (x_arr**2 + y_arr**2)
vmin = min(color_var_arr) - 0.2*(max(color_var_arr) - min(color_var_arr))
vmax = max(color_var_arr) + 0.1*(max(color_var_arr) - min(color_var_arr))
cmap = 'Paired'#'tab10'#'jet'

for lid, sid in author_lid_to_sid_dict.items():
        xa, ya = author_sid_to_xy[sid]
        auth_data = author_sid_to_name_dict[sid]
        author, _, papers_total =  pcn.get_name_for_author(auth_data=auth_data)
        author_txt = pcn.abbriviate_author_name(auth_name=author)
        color_var = (xa**2 + ya**2 )


        text_rot = np.arctan2(ya,xa)*180/np.pi
        if text_rot > 90:
            text_rot = text_rot - 180
        if text_rot < -90:
            text_rot = text_rot + 180 

        if author not in authors_to_exclude:
            if author in authors_special:
                axs.scatter(xa, ya, s= papers_total, 
                                    c=color_var, 
                                    cmap=cmap, vmin=vmin, vmax=vmax)   
                
                axs.text(   xa, ya-6, s= author_txt, 
                                    fontsize= 16,
                                    horizontalalignment='center', 
                                    verticalalignment='bottom',
                                    alpha= 1,
                                    rotation=text_rot*0, 
                                    color = "darkslateblue",
                                    weight='bold')

                for coauth_lid, coauthor_conn in enumerate(coauthor_network[lid]):
                    names_conn = author_sid_to_name_dict[author_lid_to_sid_dict[coauth_lid]]
                    flag_conn = 1
                    for name_c in names_conn:
                        if name_c in authors_to_exclude:
                            flag_conn = 0
                            break

                    if coauthor_conn > 0 and flag_conn:
                        xb, yb = author_sid_to_xy[author_lid_to_sid_dict[coauth_lid]]
                        axs.plot([xa, xb], [ya, yb], 'k', alpha=0.2, zorder=-1, linewidth=0.1)

            elif (author in authors_to_emphasize):

                axs.scatter(xa, ya, s= 10+papers_total*1, 
                                    c=color_var, 
                                    cmap=cmap, vmin=vmin, vmax=vmax)   
                
                axs.text(   xa, ya, s= author_txt, 
                                    fontsize= 12,
                                    horizontalalignment='center', 
                                    verticalalignment='bottom',
                                    alpha= 1,
                                    rotation=text_rot,
                                    color = 'brown')
                for coauth_lid, coauthor_conn in enumerate(coauthor_network[lid]):
                    names_conn = author_sid_to_name_dict[author_lid_to_sid_dict[coauth_lid]]
                    flag_conn = 1
                    for name_c in names_conn:
                        if name_c in authors_to_exclude:
                            flag_conn = 0
                            break

                    if coauthor_conn > 0 and flag_conn:
                        xb, yb = author_sid_to_xy[author_lid_to_sid_dict[coauth_lid]]
                        axs.plot([xa, xb], [ya, yb], 'k', alpha=0.2, zorder=-1, linewidth=0.1)
            else:
                axs.scatter(xa, ya, s=papers_total*4, 
                                    c=color_var, cmap=cmap, vmin=vmin, vmax=vmax)
                axs.text(xa, ya, s=author_txt, 
                                    fontsize= 9,#4+ np.log(n_colabs)*1.,
                                    horizontalalignment='center', 
                                    verticalalignment='bottom',
                                    alpha=0.9, rotation=text_rot)
                
                for coauth_lid, coauthor_conn in enumerate(coauthor_network[lid]):
                    names_conn = author_sid_to_name_dict[author_lid_to_sid_dict[coauth_lid]]
                    flag_conn = 1
                    for name_c in names_conn:
                        if name_c in authors_to_exclude:
                            flag_conn = 0
                            break

                    if coauthor_conn > 0 and flag_conn:
                        xb, yb = author_sid_to_xy[author_lid_to_sid_dict[coauth_lid]]
                        axs.plot([xa, xb], [ya, yb], 'k', alpha=0.2, zorder=-1, linewidth=0.1)
fig.tight_layout()
fig.savefig("coauthor_network.png", dpi=300)
    